# Set Up Databases and Populating Tickers and Historical Stock Data

# 1. Setup & Imports

In [2]:
import sys
import os
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

from src.db_schema import DATABASES
from src.etl.populate_tickers import recreate_database, populate_tickers
from src.etl.populate_prices import populate_prices
from src.utils.db_utils import fetch_all_asset_metadata, fetch_all_asset_prices

Project root added to sys.path: d:\dev\stat_656_autotrader


# 2. Initialize SQLite Databases

In [3]:
DB_DIR.mkdir(exist_ok=True)

def setup_databases():
    """Initialize databases and tables based on defined schemas."""
    for db_name, schemas in DATABASES.items():
        db_path = DB_DIR / db_name

        try:
            with sqlite3.connect(db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON;")
                cursor = conn.cursor()
                for schema in schemas:
                    cursor.execute(schema)
                conn.commit()
                print(f"✅ Tables created/verified in '{db_name}'")
        except sqlite3.Error as e:
            print(f"❌ Error setting up '{db_name}': {e}")

# Run database setup
setup_databases()

✅ Tables created/verified in 'assets.db'
✅ Tables created/verified in 'portfolio_management.db'
✅ Tables created/verified in 'accounting.db'
✅ Tables created/verified in 'modeling.db'
✅ Tables created/verified in 'exogenous.db'
✅ Tables created/verified in 'db_change_log.db'


# 3. Populate Asset Metadata (Tickers)

In [4]:
print("\n🔄 Recreating and populating tickers database...")
recreate_database()
populate_tickers()
print("✅ Tickers populated successfully.")


🔄 Recreating and populating tickers database...
✅ Tickers populated successfully.


In [5]:
# View the asset metadata
asset_metadata_df = fetch_all_asset_metadata()

print("DataFrame shape:", asset_metadata_df.shape)
print("DataFrame columns:", asset_metadata_df.columns.tolist())
display(asset_metadata_df.head(20))

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Raw rows fetched: [(1, 'PAMT', 'PAMT CORP Common Stock', 'NASDAQ', 'us_equity', 1, '2025-04-16', None, '2025-04-16 04:41:04'), (2, 'ADNWW', 'Advent Technologies Holdings, Inc. Warrant', 'NASDAQ', 'us_equity', 1, '2025-04-16', None, '2025-04-16 04:41:04'), (3, 'ADN', 'Advent Technologies Holdings, Inc. Class A Common Stock', 'NASDAQ', 'us_equity', 1, '2025-04-16', None, '2025-04-16 04:41:04'), (4, 'AMTX', 'Aemetis, Inc. (DE) Common Stock', 'NASDAQ', 'us_equity', 1, '2025-04-16', None, '2025-04-16 04:41:04'), (5, 'OXSQG', 'Oxford Square Capital Corp. 5.50% Notes due 2028', 'NASDAQ', 'us_equity', 1, '2025-04-16', None, '2025-04-16 04:41:04')]
Column names from DB: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed', 'fetched_at']
DataFrame shape: (7458, 9)
DataFrame columns: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed',

,asset_id,symbol,name,exchange,asset_type,is_active,date_added,date_removed,fetched_at
0,1,PAMT,PAMT CORP Common Stock,NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
1,2,ADNWW,"Advent Technologies Holdings, Inc. Warrant",NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
2,3,ADN,"Advent Technologies Holdings, Inc. Class A Com...",NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
3,4,AMTX,"Aemetis, Inc. (DE) Common Stock",NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
4,5,OXSQG,Oxford Square Capital Corp. 5.50% Notes due 2028,NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
5,6,PHGE,BiomX Inc.,AMEX,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
6,7,PRFX,PainReform Ltd. Ordinary Shares,NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
7,8,PRFZ,Invesco Exchange-Traded Fund Trust Invesco RAF...,NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
8,9,PRH,"Prudential Financial, Inc. 5.950% Junior Subor...",NYSE,us_equity,1,2025-04-16,None,2025-04-16 04:41:04
9,10,PRME,"Prime Medicine, Inc. Common Stock",NASDAQ,us_equity,1,2025-04-16,None,2025-04-16 04:41:04


In [6]:
asset_metadata_df['exchange'].unique()

array(['NASDAQ', 'AMEX', 'NYSE'], dtype=object)

# 4. Populate Historical Stock Prices

In [7]:
# Ensure the historical prices are populated. 
populate_prices()
print("✅ Historical stock prices populated successfully.")

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db


Alpaca Download Progress: 100%|██████████| 7458/7458 [2:59:21<00:00,  1.44s/it]  


Processed 7458 tickers, missing data for 27 tickers.
Total time: 10761.55 seconds. Avg time per ticker: 1.44 seconds.
✅ Historical stock prices populated successfully.


In [ ]:
# Fetch and display the historical prices
print("\n📊 Fetching and displaying historical stock prices...")
prices_df = fetch_all_asset_prices()

# Display basic info and the first 20 rows
print(f"DataFrame shape: {prices_df.shape}")
print(f"Columns: {prices_df.columns.tolist()}")
display(prices_df.head(20))